In [30]:
import os
import shutil
import nibabel as nib
import numpy as np
from glob import glob
import SimpleITK as sitk

source = glob('/media/sc-mlsg/T7/Parkinson_data_copy/**/*.nii*')
print(len(source))

326


In [34]:
# check same patient has same img orientation
PATH = '/media/sc-mlsg/T7/Parkinson_data_copy/'
pt_lst = []
for (path, dirs, files) in os.walk(PATH):
    if files != []:
        pt = path.split(os.path.sep)[-1]
        for file in files:
            # load img
            img = nib.load(os.path.join(path, file))
           # print(file)
            if nib.aff2axcodes(img.affine) != ('L', 'P', 'S'):
                print(nib.aff2axcodes(img.affine))            
                print(file)
                pt_lst.append(pt)
    print()

print(set(pt_lst))








('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz



('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz






('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz





('P', 'I', 'L')
T2F.nii.gz



('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz


('L', 'I', 'P')
T1.nii.gz



('P', 'I', 'L')
T2F.nii.gz

('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz



('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz





('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz






('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz



('P', 'I', 'L')
T2F.nii.gz


('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz










('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz









('P', 'I', 'L')
T2F.nii.gz













('L', 'I', 'P')
T1.nii.gz


















('L', 'I', 'P')
T1.nii.gz







('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz


('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz







('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
T2F.nii.gz

('P', 'I',

In [2]:
# check is tumor mask orientation match img orientation
PATH = '/media/sc-mlsg/T7/Parkinson_data_copy/'
maskPATH = '/home/sc-mlsg/Downloads/20210422_IDHwtLGG_re_added_preprocessed/TCGA/00_Tumor_masks_resampled/'
pt_lst = []
for (path, dirs, files) in os.walk(PATH):
    if files != []:
        pt = path.split(os.path.sep)[-1]
        mask = nib.load(os.path.join(maskPATH, pt, '{}_mask.nii.gz'.format(pt)))
        sitk_img = sitk.ReadImage(os.path.join(maskPATH, pt, '{}_mask.nii.gz'.format(pt)))
        mask_dirc = np.array(sitk_img.GetDirection()).reshape(3,3)
        print('mask_affine:', nib.aff2axcodes(mask.affine))

        for file in files:
            # load img
            img = nib.load(os.path.join(path, file))
           # print(file)
            sitk_img = sitk.ReadImage(os.path.join(path, file))
            if nib.aff2axcodes(mask.affine) != nib.aff2axcodes(img.affine):
                print(file) 
                print('mask_affine:', nib.aff2axcodes(mask.affine))
                print('mr_affine:', nib.aff2axcodes(img.affine))
            if not np.array_equal(np.round(mask_dirc,5), np.round(np.array(sitk_img.GetDirection()).reshape(3,3),5)):
                print(file)
                print('mask_direction:\n', np.array(sitk_img.GetDirection()).reshape(3,3))
                print('mr_direction:\n',np.array(sitk_img.GetDirection()).reshape(3,3))
                pt_lst.append(pt)
    print()

print(set(pt_lst))


FileNotFoundError: No such file or no access: '/home/sc-mlsg/Downloads/20210422_IDHwtLGG_re_added_preprocessed/TCGA/00_Tumor_masks_resampled/8403960/8403960_mask.nii.gz'

In [29]:
# reorient both image and header
total = 0
lst = []
for filepath in source:
    total+=1
    # load img
    img = nib.load(filepath)
    print(nib.aff2axcodes(img.affine))
    
    # only when the image orientation is PIL -> PSL
    if nib.aff2axcodes(img.affine)[1] == 'I':
        print(filepath.split(os.path.sep)[-1])
        # print orientation of original img
        print(nib.aff2axcodes(img.affine))
        sitk_img = sitk.ReadImage(filepath)
        print(np.array(sitk_img.GetDirection()).reshape(3,3))
        
        # reorient img(use np.flip or np.transpose)
        img_data = sitk.GetArrayFromImage(sitk_img)
        img_T = np.flip(img_T, axis=1)
        
        # reset image header info
        flipFilter = sitk.FlipImageFilter()
        
        flipAxes = (False, True, False) #(width, height, depth)
        flipFilter.SetFlipAxes(flipAxes)
        
        new_img = flipFilter.Execute(sitk_img)
        
        # save new header info
        ni_img = sitk.GetImageFromArray(img_T)
        ni_img.SetOrigin(new_img.GetOrigin())
        ni_img.SetSpacing(new_img.GetSpacing())
        ni_img.SetDirection(new_img.GetDirection())
        
        # save file
        outpath = os.path.join(os.path.dirname(filepath), os.path.basename(filepath).split('.')[0]+'_flipped.nii.gz')
        
        sitk.WriteImage(ni_img, outpath)
        
        # check orientation of reoriented img
        img = nib.load(outpath)
        print(nib.aff2axcodes(img.affine))
        
        sitk_mask = sitk.ReadImage(outpath)
        print(np.array(sitk_mask.GetDirection()).reshape(3,3))
        

('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
[[-0.05390628  0.00515339  0.9985327 ]
 [ 0.98995183  0.13119115  0.05276596]
 [ 0.13072673 -0.9913437   0.01217363]]
('P', 'S', 'L')
[[-0.05390628 -0.00515334  0.9985327 ]
 [ 0.98995184 -0.1311911   0.05276597]
 [ 0.13072668  0.9913437   0.01217358]]
('P', 'I', 'L')
T2F.nii.gz
('P', 'I', 'L')
[[-0.05390628  0.00515338  0.99853268]
 [ 0.98995183  0.1311912   0.05276571]
 [ 0.13072678 -0.99134369  0.01217648]]
('P', 'S', 'L')
[[-0.05390637 -0.00515487  0.99853269]
 [ 0.98995183 -0.13119124  0.05276586]
 [ 0.13072674  0.99134368  0.01217511]]
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
[

('P', 'I', 'L')
T2F.nii.gz
('P', 'I', 'L')
[[-0.06024179  0.01185563  0.99811339]
 [ 0.99507014 -0.07820485  0.06098733]
 [-0.07878034 -0.99686681  0.00708584]]
('P', 'S', 'L')
[[-0.06024191 -0.01185554  0.9981134 ]
 [ 0.99507012  0.07820495  0.06098714]
 [-0.07878045  0.99686681  0.00708588]]
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
[[ 0.08279077  0.01398759  0.99646878]
 [ 0.97620426 -0.2022358  -0.0782683 ]
 [-0.20042688 -0.97923696  0.03039801]]
('P', 'S', 'L')
[[ 0.08279077 -0.01398753  0.99646878]
 [ 0.97620424  0.20223587 -0.07826831]
 [-

('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('P', 'I', 'L')
T2F.nii.gz
('P', 'I', 'L')
[[ 0.03917273  0.03109663  0.99874853]
 [ 0.97969438 -0.19788364 -0.03226375]
 [-0.1966327  -0.97973214  0.03821563]]
('P', 'S', 'L')
[[ 0.03917235 -0.03109607  0.9987485 ]
 [ 0.9796944   0.1978836  -0.03226391]
 [-0.19663268  0.97973216  0.03821622]]
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
(

('P', 'S', 'L')
[[ 0.02111427  0.02418072  0.99948461]
 [ 0.96827927  0.24846487 -0.02646621]
 [-0.24897677  0.96833904 -0.01816753]]
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('

[[-3.39380646e-03 -3.14099028e-02  9.99500825e-01]
 [ 9.94213497e-01 -1.07422175e-01  4.85479575e-08]
 [ 1.07368544e-01  9.93717210e-01  3.15927197e-02]]
('P', 'I', 'L')
T2F.nii.gz
('P', 'I', 'L')
[[-0.00339375  0.03140988  0.9995008 ]
 [ 0.9942135   0.10742215  0.        ]
 [ 0.10736852 -0.99371721  0.03159347]]
('P', 'S', 'L')
[[-3.39388187e-03 -3.14103106e-02  9.99500812e-01]
 [ 9.94213498e-01 -1.07422166e-01  7.98497454e-08]
 [ 1.07368534e-01  9.93717199e-01  3.15931307e-02]]
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('L', 'P', 'S')
('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
[[ 9.27528190e-05  2.39111086e-03  9.99997137e-01]
 [ 9.99249670e-01 -3.87310814e-02 -7.29181123e-08]
 [-3.87309707e-02 -9.99246809e-01  2.39290902e-03]]
('P', 'S', 'L')
[[ 9.27528191e-05 -2.39113188e-03  9.99997137e-01]
 [ 9.99249671e-01  3.87310602e-02 -7.21474932e-08]
 [-3.87309495e-02  9.99246810e-01  2.39293021e-03]]
('P', 'I', 'L')
T2F.nii.gz
('P', 'I', 'L')
[[ 9.26764451e-05  2.39101837e-03  9.99997

('P', 'S', 'L')
[[-3.50859873e-02 -1.10958770e-02  9.99322698e-01]
 [ 9.52811240e-01  3.01324759e-01  3.67987076e-02]
 [-3.01528960e-01  9.53457011e-01  5.76072515e-10]]
('P', 'I', 'L')
T2F.nii.gz
('P', 'I', 'L')
[[-0.03508599  0.01109588  0.99932271]
 [ 0.95281126 -0.30132468  0.03679826]
 [-0.30152891 -0.95345704 -0.        ]]
('P', 'S', 'L')
[[-3.50858385e-02 -1.10958178e-02  9.99322704e-01]
 [ 9.52811284e-01  3.01324623e-01  3.67985517e-02]
 [-3.01528840e-01  9.53457054e-01 -7.27775711e-09]]


In [15]:
# reorient both image and header
total = 0
lst = []
for filepath in source[:1]:
    total+=1
    # load img
    img = nib.load(filepath)
    print(nib.aff2axcodes(img.affine))
    
    # only when the image orientation is PIL, LIP -> LPS
    if nib.aff2axcodes(img.affine)[0] == 'P':
        print(filepath.split(os.path.sep)[-1])
        # print orientation of original img
        print(nib.aff2axcodes(img.affine))
        sitk_img = sitk.ReadImage(filepath)
        print(np.array(sitk_img.GetDirection()).reshape(3,3))
        
        # reorient img(use np.flip or np.transpose)
        img_data = sitk.GetArrayFromImage(sitk_img)
        img_T = np.flip(img_data, axis=1)
        img_T = img_T.transpose(0, 2, 1)
        
        # reset image header info
        flipFilter = sitk.FlipImageFilter()
        
        flipAxes = (False, True, False) #(width, height, depth)
        flipFilter.SetFlipAxes(flipAxes)
        
        temp_img = flipFilter.Execute(sitk_img)
        
        new_img = sitk.PermuteAxes(temp_img, [0,2,1])
        
        # save new header info
        ni_img = sitk.GetImageFromArray(img_T)
        ni_img.SetOrigin(new_img.GetOrigin())
        ni_img.SetSpacing(new_img.GetSpacing())
        ni_img.SetDirection(new_img.GetDirection())
        
        # save file
        outpath = os.path.join(os.path.dirname(filepath), os.path.basename(filepath).split('.')[0]+'_flipped.nii.gz')
        
        sitk.WriteImage(ni_img, outpath)
        
        # check orientation of reoriented img
        img = nib.load(outpath)
        print(nib.aff2axcodes(img.affine))
        
        sitk_mask = sitk.ReadImage(outpath)
        print(np.array(sitk_mask.GetDirection()).reshape(3,3))

    # only when the image orientation is PIL, LIP -> LPS
    if nib.aff2axcodes(img.affine)[-1] == 'P':
        print(filepath.split(os.path.sep)[-1])
        # print orientation of original img
        print(nib.aff2axcodes(img.affine))
        sitk_img = sitk.ReadImage(filepath)
        print(np.array(sitk_img.GetDirection()).reshape(3,3))
        
        # reorient img(use np.flip or np.transpose)
        img_data = sitk.GetArrayFromImage(sitk_img)
        img_T = np.flip(img_data, axis=1)
        img_T = img_T.transpose(0, 2, 1)
        
        # reset image header info
        flipFilter = sitk.FlipImageFilter()
        
        flipAxes = (False, True, False) #(width, height, depth)
        flipFilter.SetFlipAxes(flipAxes)
        
        temp_img = flipFilter.Execute(sitk_img)
        
        new_img = sitk.PermuteAxes(temp_img, [0,2,1])
        
        # save new header info
        ni_img = sitk.GetImageFromArray(img_T)
        ni_img.SetOrigin(new_img.GetOrigin())
        ni_img.SetSpacing(new_img.GetSpacing())
        ni_img.SetDirection(new_img.GetDirection())
        
        # save file
        outpath = os.path.join(os.path.dirname(filepath), os.path.basename(filepath).split('.')[0]+'_flipped.nii.gz')
        
        sitk.WriteImage(ni_img, outpath)
        
        # check orientation of reoriented img
        img = nib.load(outpath)
        print(nib.aff2axcodes(img.affine))
        
        sitk_mask = sitk.ReadImage(outpath)
        print(np.array(sitk_mask.GetDirection()).reshape(3,3))


('P', 'I', 'L')
T1.nii.gz
('P', 'I', 'L')
[[-3.50859873e-02  1.10959041e-02  9.99322698e-01]
 [ 9.52811240e-01 -3.01324732e-01  3.67987151e-02]
 [-3.01528960e-01 -9.53457019e-01  2.71287078e-08]]
('P', 'L', 'S')
[[-3.50859099e-02  9.99322695e-01 -1.10963646e-02]
 [ 9.52811382e-01  3.67987820e-02  3.01324272e-01]
 [-3.01528520e-01  5.07227801e-07  9.53457159e-01]]


In [ ]:
# reorient both image and header
total = 0
lst = []
for filepath in source:
    total+=1
    # load img
    img = nib.load(filepath)
    
    # only when pt is in pt_lst
    if pt in pt_lst:
        print(filepath.split(os.path.sep)[-1])
        # print orientation of original img
        print(nib.aff2axcodes(img.affine))
        sitk_img = sitk.ReadImage(filepath)
        print(np.array(sitk_img.GetDirection()).reshape(3,3))
        
        # reorient img(use np.flip or np.transpose)
        img_data = sitk.GetArrayFromImage(sitk_img)
        img_T = np.flip(img_data, axis=1)
        
        # reset image header info
        flipFilter = sitk.FlipImageFilter()
        
        flipAxes = (False, True, False) #(width, height, depth)
        flipFilter.SetFlipAxes(flipAxes)
        
        new_img = flipFilter.Execute(sitk_img)
        
        # save new header info
        ni_img = sitk.GetImageFromArray(img_T)
        ni_img.SetOrigin(new_img.GetOrigin())
        ni_img.SetSpacing(new_img.GetSpacing())
        ni_img.SetDirection(new_img.GetDirection())
        
        # save file
        outpath = os.path.join(os.path.dirname(filepath), os.path.basename(filepath).split('.')[0]+'_flipped.nii')
        
  #      sitk.WriteImage(ni_img, outpath)
        
        # check orientation of reoriented img
  #      img = nib.load(outpath)
  #      print(nib.aff2axcodes(img.affine))
        
  #      sitk_mask = sitk.ReadImage(outpath)
  #      print(np.array(sitk_mask.GetDirection()).reshape(3,3))
        